In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".", ",", "'", ":", "(", ")", "-"]

In [6]:
for intent in intents["intents"]:
    for pattern in intent["symptoms"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
            
print(documents)

[(['fever'], 'flu'), (['chills'], 'flu'), (['cough'], 'flu'), (['sore', 'throat'], 'flu'), (['runny', 'or', 'stuffy', 'nose'], 'flu'), (['muscle', 'or', 'body', 'aches'], 'flu'), (['headaches'], 'flu'), (['fatigue'], 'flu'), (['nausea'], 'flu'), (['vomiting'], 'flu'), (['diarrhea'], 'flu'), (['you', 'may', 'also', 'experience', 'a', 'mild', 'headache', ',', 'body', 'aches', 'or', 'a', 'low', 'grade', 'fever', '.', 'typically', ',', 'a', 'cold', 'lasts', '2-14', 'days', '.'], 'flu'), (['flu', 'symptoms', 'come', 'on', 'very', 'quickly', 'and', 'can', 'include', ':', 'a', 'sudden', 'high', 'temperature', ',', 'an', 'aching', 'body', ',', 'feeling', 'tired', 'or', 'exhausted', ',', 'a', 'dry', 'cough', ',', 'a', 'sore', 'throat', ',', 'a', 'headache', ',', 'difficulty', 'sleeping', ',', 'loss', 'of', 'appetite', ',', 'diarrhea', 'or', 'tummy', 'pain', ',', 'feeling', 'sick', 'and', 'being', 'sick'], 'flu'), (['eye', 'redness'], 'conjunctivitis'), (['eye', 'itching'], 'conjunctivitis'), ([

In [7]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
print(words)

["'re", '12-14', '2-14', 'The', 'When', 'a', 'abdomen', 'about', 'ache', 'aching', 'acting', 'activity', 'acute', 'adult', 'after', 'agitated', 'alcohol', 'all', 'allergic', 'also', 'although', 'an', 'and', 'and/or', 'animal', 'antacid', 'antibiotic', 'anxious', 'aphasia', 'appear', 'appearing', 'appetite', 'are', 'area', 'arm', 'around', 'asking', 'asthma', 'at', 'atopic', 'attack', 'attention', 'back', 'bacterial', 'be', 'because', 'become', 'behaviour', 'behind', 'being', 'believing', 'belly', 'blister', 'blistering', 'bloating', 'blocked', 'blood', 'bloody', 'body', 'bowel', 'breathing', 'burning', 'but', 'button', 'by', 'ca', 'can', 'careless', 'carers', 'carry', 'cause', 'caused', 'change', 'changing', 'chemical', 'chest', 'child', 'chill', 'chlorine', 'clear', 'cluster', 'cold', 'come', 'common', 'commonly', 'confusion', 'conjunctiva', 'conjunctivitis', 'constant', 'constantly', 'containing', 'contaminated', 'continuous', 'conversation', 'cough', 'coughing', 'could', 'cover', 'c

In [8]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [9]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [10]:
random.shuffle(training)
training = np.array(training)

X_train = list(training[:, 0])
y_train = list(training[:, 1])

C:\Users\hasan\AppData\Local\Temp/ipykernel_8144/633689806.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation="softmax"))

adam = Adam(learning_rate=0.001, decay=1e-6)

model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])
history = model.fit(np.array(X_train), np.array(y_train), 
                    epochs=100, batch_size=16, verbose=1)

Epoch 1/100
7/7 [==============================] - 2s 6ms/step - loss: 2.3948 - accuracy: 0.1238
Epoch 2/100
7/7 [==============================] - 0s 7ms/step - loss: 2.3695 - accuracy: 0.1810
Epoch 3/100
7/7 [==============================] - 0s 7ms/step - loss: 2.3132 - accuracy: 0.1810
Epoch 4/100
7/7 [==============================] - 0s 7ms/step - loss: 2.3157 - accuracy: 0.1810
Epoch 5/100
7/7 [==============================] - 0s 7ms/step - loss: 2.2880 - accuracy: 0.2381
Epoch 6/100
7/7 [==============================] - 0s 7ms/step - loss: 2.2426 - accuracy: 0.2476
Epoch 7/100
7/7 [==============================] - 0s 5ms/step - loss: 2.2230 - accuracy: 0.2571
Epoch 8/100
7/7 [==============================] - 0s 7ms/step - loss: 2.1745 - accuracy: 0.2952
Epoch 9/100
7/7 [==============================] - 0s 7ms/step - loss: 2.1502 - accuracy: 0.2952
Epoch 10/100
7/7 [==============================] - 0s 7ms/step - loss: 2.1210 - accuracy: 0.3714
Epoch 11/100
7/7 [===========

In [12]:
model.save("healthbot_model.h5", history)